# Connect to Workspace

Connect to your AML workspace.

In [ ]:
from azureml.core.workspace import Workspace

try:    
    # Get instance of the Workspace and write it to config file
    ws = Workspace(
        subscription_id = '<subscription_id>', 
        resource_group = '<resource_group>', 
        workspace_name = '<workspace_name>')

    # Writes workspace config file
    ws.write_config()
    
    print('Library configuration succeeded')
except Exception as e:
    print(e)
    print('Workspace not found')

# Upload files to datastore

Load the datasets into blob storage.

In [ ]:
ds = ws.get_default_datastore()
ds.upload('../../azureStorageFiles')

# Load files to tabular Dataset

A range of datasets are used to support different parts of the demo.

In [9]:
from azureml.core import Dataset

airports = Dataset.Tabular.from_delimited_files(path=[(ds, 'airports.csv')])
carriersdataset = Dataset.Tabular.from_delimited_files(path=[(ds, 'carriersdataset.csv')])
flight_dataset_2008 = Dataset.Tabular.from_delimited_files(path=[(ds, 'flight_dataset_2008.csv')])
plane_data = Dataset.Tabular.from_delimited_files(path=[(ds, 'plane-data.csv')])
flightdelayweather_ds_clean = Dataset.Tabular.from_delimited_files(path=[(ds, 'flightdelayweather_ds_clean.csv')])
flight_dataset_2008_with_weather = Dataset.Tabular.from_delimited_files(path=[(ds, 'flight_dataset_2008_with_weather.csv')])

# Register Datasets to Workspace

By registering the datasets, we can link experiment runs to them throughout the demo series.

In [11]:
airports_ds = airports.register(workspace=ws, name='airports_ds', create_new_version=True)
flightdelayweather_ds = flight_dataset_2008_with_weather.register(workspace=ws, name='flightdelayweather_ds', create_new_version=True)
carriers_ds = carriersdataset.register(workspace=ws, name='carriers_ds', create_new_version=True)
flightdelay_ds_raw = flight_dataset_2008.register(workspace=ws, name='flightdelay_ds_raw', create_new_version=True)
flightdelay_ds = flight_dataset_2008_with_weather.register(workspace=ws, name='flightdelay_ds', create_new_version=True)
flight_dataset_2008_with_weather = flight_dataset_2008_with_weather.register(workspace=ws, name='flight_dataset_2008_with_weather', create_new_version=True)
flightdelayweather_ds_clean = flightdelayweather_ds_clean.register(workspace=ws, name='flightdelayweather_ds_clean', create_new_version=True)


# Build Docker Image for Flight Delay DL Notebook

A custom Docker image has been prepared for the deep learning scenario. It contains additional components for building the object detection model.

Make sure to update your container registry name below before running the command.

In [ ]:
!az acr build --image aml/dl-image:v1 --registry [your registry name] --file ../../docker/Dockerfile ../../docker


# Create AML Cluster

A CPU cluster is recommended for remote training.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

### Create AML CPU Compute Cluster

try:
    compute_target = ComputeTarget(workspace=ws, name='cpucluster')
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS12_v2',
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, 'cpucluster', compute_config)

    compute_target.wait_for_completion(show_output=True)

# Create GPU Cluster

A GPU cluster is recommended for the deep learning scenario.

In [ ]:
# Choose a name for your GPU cluster
cluster_name = "gpucluster"

try:
    gpu_cluster = ComputeTarget(workspace = ws, name = cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6',
                                                           min_nodes=0,
                                                           max_nodes=4,
                                                           admin_username="theadmin",
                                                           admin_user_password="Password123")
                                                           
    gpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

gpu_cluster.wait_for_completion(show_output=True)

# Create simple AutoML Run

This run will ensure there is run history when showcasing the AutoML UX. It will also be called from Power BI (see next step).

In [ ]:
from azureml.train.automl import AutoMLConfig
from azureml.core.experiment import Experiment
from azureml.train.automl.run import AutoMLRun 
from azureml.widgets import RunDetails
import os

project_folder = './scripts'
# Working directory
if not os.path.exists(project_folder):
    os.makedirs(project_folder)

tabular = Dataset.get_by_name(ws, 'flightdelayweather_ds')

data = tabular.to_pandas_dataframe()
training_data, validation_data = tabular.random_split(percentage=0.9, seed=1)

automl_config = AutoMLConfig(task = 'classification',
                             iterations = 3,
                             iteration_timeout_minutes = 30, 
                             max_cores_per_iteration = 10,
                             primary_metric = 'accuracy',
                             debug_log = 'automl.log',
                             training_data = training_data,
                             validation_data = validation_data,
                             label_column_name = "ArrDelay15",
                             compute_target = compute_target,
                             path = project_folder,
                             model_explainability = True,
                             experiment_exit_score = 0.8,
                             enable_early_stopping = True,
                             enable_onnx_compatible_models=True)

# Get an instance of the Workspace from the config file
ws = Workspace.from_config()

# Create Experiment
experiment = Experiment(ws, 'flight-delay-exp')
remote_run = experiment.submit(automl_config, show_output=False)
remote_run = AutoMLRun(experiment=experiment, run_id=remote_run.id)
RunDetails(remote_run).show()


# Deploy to ACI Instance

This service will be used by Power BI for inferencing.

In [ ]:
%%writefile score-new.yml
name: project_environment
dependencies:
- python=3.6.2

- pip:
  - azureml-sdk[notebooks,automl]~=1.32.0
  - azureml-defaults~=1.32.0
  - inference-schema
  - azureml-monitoring
  - shap==0.39.0
- numpy
- scikit-learn==0.22.1
channels:
- anaconda
- conda-forge

In [ ]:
%%writefile score.py
import os
import pickle
import json
import joblib
import time
import numpy as np
import pandas as pd
import azureml.automl.core
from azureml.core.model import Model
 
def init():
    global model
    print ("model initialized" + time.strftime("%H:%M:%S"))
    model_path = os.path.join(os.getenv("AZUREML_MODEL_DIR"), "model.pkl")
    model = joblib.load(model_path)
    
def run(data):
    try:
        data = json.loads(data)["data"]
        df = pd.DataFrame(data, columns=['Month', 'DayofMonth', 'DayOfWeek', 'CRSDepTime', 'CRSArrTime', 'UniqueCarrier', 'CRSElapsedTime', 'Origin', 'Dest', 'Distance', 'Origin_Lat', 'Origin_Lon', 'Origin_State', 'Dest_Lat', 'Dest_Lon', 'Dest_State', 'Origin_dayl', 'Dest_dayl', 'Origin_prcp', 'Dest_prcp', 'Origin_srad', 'Dest_srad', 'Origin_swe', 'Dest_swe', 'Origin_tmax', 'Dest_tmax', 'Origin_tmin', 'Dest_tmin', 'Origin_vp', 'Dest_vp']) 
        result = model.predict(df)
    except Exception as e:
        result = str(e)
        return {"error": result}
    return {"result":result.tolist()}

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model, InferenceConfig

remote_run.wait_for_completion()
best_run, fitted_model = remote_run.get_output()
model = best_run.register_model(model_name='flight_delay_weather', 
                                model_path='outputs/model.pkl')

inference_config = InferenceConfig(runtime= "python",
                                    entry_script="score.py",
                                    conda_file="score-new.yml")

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "aciservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)